In [15]:
import pandas as pd
import numpy as np
import os as os
import aux.acessos as ac
import aux.aed as aed
import pickle
import statsmodels.api as sm
from statsmodels.formula.api import ols
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import random
from apyori import apriori

In [14]:
#!pip install apyori

  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=7ba35ec436a8e7d93954d1510af2dfe25973c982dabc676473c8b994c27cee6a
  Stored in directory: /home/flavia.costa/.cache/pip/wheels/1b/02/6c/a45230be8603bd95c0a51cd2b289aefdd860c1a100eab73661
Successfully built apyori


In [12]:

# Query
query = """

with kpis as (
    select 
        dt_chat,
        count(distinct info_cliente.id_issue) as chats
        , count(distinct case when issues_macro.is_recontato_2d = 1 then info_cliente.id_issue end) / cast(count(distinct info_cliente.id_issue) as double) as tx_recontato
        , avg(issues_macro.nr_tma_resolucao) as nr_tma_resolucao 
        , avg(issues_macro.nr_tpr) as nr_tpr
        , count(distinct case when novas_tags.fila_de_atendimento = 'Pagamento' then info_cliente.id_issue end) / cast(count(distinct info_cliente.id_issue) as double)
        as p_pagamento
        , count(distinct case when novas_tags.fila_de_atendimento = 'Fatura' then info_cliente.id_issue end) / cast(count(distinct info_cliente.id_issue) as double)
        as p_fatura
        , count(distinct case when novas_tags.fila_de_atendimento = 'Fraude' then info_cliente.id_issue end) / cast(count(distinct info_cliente.id_issue) as double)
        as p_fraude
        , count(distinct case when novas_tags.fila_de_atendimento = 'Logistica' then info_cliente.id_issue end) / cast(count(distinct info_cliente.id_issue) as double)
        as p_logistica
        , count(distinct case when novas_tags.fila_de_atendimento = 'Geral' and novas_tags.hierarquia_1 = 'Cartão' then info_cliente.id_issue end) / cast(count(distinct info_cliente.id_issue) as double)
        as p_geral_cartao
        , count(distinct case when novas_tags.fila_de_atendimento = 'Geral' and novas_tags.hierarquia_1 = 'Conta' then info_cliente.id_issue end) / cast(count(distinct info_cliente.id_issue) as double)
        as p_geral_conta
        , count(distinct case when novas_tags.fila_de_atendimento = 'Cadastro' then info_cliente.id_issue end) / cast(count(distinct info_cliente.id_issue) as double)
        as p_cadastro
        , count(distinct case when novas_tags.fila_de_atendimento = 'Geral' and novas_tags.hierarquia_1 = 'Tecnologia' then info_cliente.id_issue end) / cast(count(distinct info_cliente.id_issue) as double)
        as p_geral_tecnologia
        , count(distinct case when is_finalizacao_automacao = 1 then info_cliente.id_issue else null end ) as qtd_encerrado_inatividade
        , avg(issues_macro.nr_csat) as nr_csat
    FROM cx_curated_zone.cx_contact_rate  AS info_cliente
    left join cx_curated_zone.helpshift_issues AS issues_macro ON info_cliente.id_issue = issues_macro.id_issue
    left join "cx_curated_zone"."helpshift_tag_niveis" AS helpshift_tag ON info_cliente."id_issue" = helpshift_tag."id_issue"
    left join processed_zone_api_cxm_tags.tags_hierarquia_gerencial as novas_tags ON trim(lower(n3)) = trim(lower(nm_tag_n3))
    Where dt_chat >= to_date('2021-01-01','yyyy-mm-dd') and dt_chat <= to_date('2022-12-24','yyyy-mm-dd')
     group by 1
     order by 1
)
, retencao as (
    select
dt_chat
, (qtd_resolvido_bot + qtd_encerrado_inatividade - qtd_reabertura) / cast(qtd_issues as double) as tx_retencao_liquida
, (qtd_resolvido_bot) / cast(qtd_issues as double) as p_resolvido_bot
from (
	select  
		cast(substring(cast(dt_criacao_chat as varchar), 1, 10) as date) as dt_chat 
		, count(distinct case when is_abandono_bot !=1 and is_resolvido_bot = 1 then id_issue else null end) as qtd_resolvido_bot
		, count(distinct case when is_abandono_bot = 1 then id_issue else null end ) as qtd_encerrado_inatividade
		, count(distinct case when is_recontato_bot>0 then id_issue else null end) as qtd_reabertura
		, count(distinct id_issue) as qtd_issues
	from cx_curated_zone.helpshift_bots
		group by 1
		order by 1
	)
)
select
k.dt_chat
, chats
, tx_recontato
, nr_tma_resolucao
, nr_tpr
, nr_csat
, qtd_encerrado_inatividade
, tx_retencao_liquida
, cast(p_pagamento * 100 as integer) as p_pagamento
, cast(p_fatura * 100 as integer) as p_fatura
, cast(p_fraude * 100 as integer) as p_fraude
, cast(p_logistica * 100 as integer) as p_logistica
, cast(p_geral_cartao * 100 as integer) as p_geral_cartao
, cast(p_geral_conta * 100 as integer) as p_geral_conta
, cast(p_cadastro * 100 as integer) as p_cadastro
, cast(p_geral_tecnologia * 100 as integer) as p_geral_tecnologia
, p_resolvido_bot
, e.is_holiday
, e.is_event
, case when e.ds_holiday_or_event is null then '0_dia_normal' else e.ds_holiday_or_event end as ds_holiday_or_event
from kpis k
left join retencao r on (r.dt_chat = k.dt_chat)
left join customer_curated_zone.ca_dict_events e on (k.dt_chat = e.dt_date)
order by 1

"""

df = ac.df_athena('flavia-costa', query)

### Tratamento de Missings

In [13]:
df.head()

,dt_chat,chats,tx_recontato,nr_tma_resolucao,nr_tpr,nr_csat,qtd_encerrado_inatividade,tx_retencao_liquida,p_pagamento,p_fatura,p_fraude,p_logistica,p_geral_cartao,p_geral_conta,p_cadastro,p_geral_tecnologia,p_resolvido_bot,is_holiday,is_event,ds_holiday_or_event
0,2021-01-01,2096,0.228053,1281.993694,1048.858515,4.451613,1372,NaN,13,9,20,11,2,1,23,0,NaN,1,0,Confraternização Universal
1,2021-01-02,4363,0.254641,1143.874720,1086.929357,4.371769,2840,NaN,13,9,16,11,1,1,28,0,NaN,0,0,0_dia_normal
2,2021-01-03,3970,0.253401,744.480124,1112.726379,4.481760,2444,NaN,13,8,14,9,1,1,32,0,NaN,0,0,0_dia_normal
3,2021-01-04,13057,0.193383,802.978337,793.419781,4.321889,7014,NaN,29,15,11,10,1,1,16,0,NaN,0,0,0_dia_normal
4,2021-01-05,8655,0.206817,1009.925391,730.438844,4.429487,5395,NaN,23,11,15,13,1,1,11,0,NaN,0,0,0_dia_normal


In [43]:
columns_categ = ['chats','nr_csat', 'p_fatura', 'is_holiday']

for column in columns_categ:
  if df[column].dtype == 'float' or df[column].dtype == 'int':
    nm_col = column + "_bin"
    df[nm_col] = pd.qcut(df[column].rank(method='first'), q=3, precision=0, labels = ['baixo', 'medio', 'alto'])
    df[nm_col] = str(nm_col) + "_" + df[nm_col].astype('string')

In [44]:
df.groupby('chats_bin').agg({'chats':['min','max']})

chats       
                   min    max
chats_bin                    
chats_bin_alto   13956  35068
chats_bin_baixo   1353   7630
chats_bin_medio   7659  13685

In [45]:
df.groupby('nr_csat_bin').agg({'nr_csat':['min','max']})

nr_csat          
                        min       max
nr_csat_bin                          
nr_csat_bin_alto   4.400313  4.644882
nr_csat_bin_baixo  3.409125  4.136576
nr_csat_bin_medio  4.137093  4.400000

In [53]:
df_assoc = df[['chats_bin','nr_csat_bin', 'p_fatura_bin','is_holiday_bin']]

transactions = []

for i in range(0, df_assoc.shape[0]):
    transactions.append([str(df_assoc.values[i, j]) for j in range(0, 2)])

print(transactions[0])

rules = apriori(transactions, min_support = 0.01, min_confidence = 0.05, min_lift = 0.5, min_length = 0)
results = list(rules)
results = pd.DataFrame(results)
results['flag_one_rule'] = np.where(results['items'].astype('str').str.contains(","), 0, 1)
results = results.query('flag_one_rule == 0')
results.head(30)

['chats_bin_baixo', 'nr_csat_bin_alto']


,items,support,ordered_statistics,flag_one_rule
6,"(nr_csat_bin_baixo, chats_bin_alto)",0.221300,"[((), (nr_csat_bin_baixo, chats_bin_alto), 0.2...",0
7,"(nr_csat_bin_medio, chats_bin_alto)",0.105118,"[((), (chats_bin_alto, nr_csat_bin_medio), 0.1...",0
8,"(nr_csat_bin_alto, chats_bin_baixo)",0.199170,"[((), (nr_csat_bin_alto, chats_bin_baixo), 0.1...",0
9,"(nr_csat_bin_medio, chats_bin_baixo)",0.116183,"[((), (nr_csat_bin_medio, chats_bin_baixo), 0....",0
10,"(nr_csat_bin_alto, chats_bin_medio)",0.127248,"[((), (nr_csat_bin_alto, chats_bin_medio), 0.1...",0
11,"(nr_csat_bin_baixo, chats_bin_medio)",0.094053,"[((), (nr_csat_bin_baixo, chats_bin_medio), 0....",0
12,"(nr_csat_bin_medio, chats_bin_medio)",0.112033,"[((), (nr_csat_bin_medio, chats_bin_medio), 0....",0


In [49]:
results.to_csv('regras.csv')